In [1]:
!pip install -q streamlit pyngrok langchain llama-cpp-python huggingface_hub langchain-community

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.9/67.9 MB 11.2 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Installing backend dependencies ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.3/44.3 kB 3.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.1/10.1 MB 54.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 66.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.5/45.5 kB 3.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.2/45.2 kB 3.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 101.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.1/79.1 kB 6.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 4.2 MB/s eta 0:00:00


In [2]:
!pip freeze > requirements.txt

In [3]:
from google.colab import files
files.download("requirements.txt")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
from huggingface_hub import hf_hub_download

# Download the GGUF model (LLaMA 2 7B Chat Q4_K_M)
model_path = hf_hub_download(
    repo_id="TheBloke/Llama-2-7B-Chat-GGUF",
    filename="llama-2-7b-chat.Q4_K_M.gguf"
)
print("✅ Downloaded model to:", model_path)

✅ Downloaded model to: /root/.cache/huggingface/hub/models--TheBloke--Llama-2-7B-Chat-GGUF/snapshots/191239b3e26b2882fb562ffccdd1cf0f65402adb/llama-2-7b-chat.Q4_K_M.gguf


In [ ]:
with open("llama_helper.py", "w") as f:
    f.write(f"""\
from langchain.llms import LlamaCpp
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain

llm = LlamaCpp(
    model_path="{model_path}",
    n_ctx=2048,
    temperature=0.7,
    n_gpu_layers=-1,  # ✅ Use all GPU layers if available (like T4)
    verbose=True,
)

prompt_template = PromptTemplate(
    input_variables=["topic"],
    template="Generate a well-structured, informative article on the topic:\\n\\n{{topic}}\\n\\nArticle:",
)

def get_article_chain():
    return LLMChain(llm=llm, prompt=prompt_template)
""")


In [ ]:
with open("llama_helper.py", "w") as f:
    f.write(f"""\
from langchain.llms import LlamaCpp
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain

llm = LlamaCpp(
    model_path="{model_path}",
    n_ctx=2048,
    temperature=0.7,
    n_gpu_layers=-1,  # ✅ Use all GPU layers if available (like T4)
    verbose=True,
)

prompt_template = PromptTemplate(
    input_variables=["topic"],
    template="Generate a well-structured, informative article on the topic:\\n\\n{{topic}}\\n\\nArticle:",
)

def get_article_chain():
    return LLMChain(llm=llm, prompt=prompt_template)
""")


In [ ]:
%%writefile main.py
import streamlit as st
from llama_helper import get_article_chain

st.set_page_config(page_title="LLaMA Article Generator", page_icon="📚")
st.title("📚 LLaMA Article Generator")

topic = st.text_input("Enter a topic for article generation:")

if topic:
    with st.spinner("Generating article..."):
        chain = get_article_chain()
        result = chain.run(topic)
        st.subheader("Generated Article")
        st.write(result)


Overwriting main.py


In [ ]:
!pkill -f streamlit || echo "No Streamlit process to kill"
from pyngrok import ngrok
ngrok.kill()

NGROK_TOKEN = input("Enter your ngrok authtoken: ").strip()
os.environ["NGROK_AUTHTOKEN"] = NGROK_TOKEN

!ngrok config add-authtoken $NGROK_AUTHTOKEN

^C
Authtoken saved to configuration file: /root/.config/ngrok/ngrok.yml


In [ ]:
from pyngrok import ngrok
import threading
import os

public_url = ngrok.connect(8501)
print(f"🌐 Streamlit app running at: {public_url}")

def run():
    os.system("streamlit run main.py")

thread = threading.Thread(target=run)
thread.start()


🌐 Streamlit app running at: NgrokTunnel: "https://8e5a-34-87-48-117.ngrok-free.app" -> "http://localhost:8501"
